# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 29 2022 10:53AM,243060,19,TR-RG-08012022,"GCH Granules USA, Inc.",Released
1,Jul 29 2022 10:52AM,243020,20,8334934,"ACI Healthcare USA, Inc.",Released
2,Jul 29 2022 10:19AM,243058,10,8338579,Beach Products Inc,Released
3,Jul 29 2022 10:15AM,243057,10,8338060,MedStone Pharma LLC,Released
4,Jul 29 2022 10:15AM,243057,10,8338061,MedStone Pharma LLC,Released
5,Jul 29 2022 10:15AM,243057,10,8338062,MedStone Pharma LLC,Released
6,Jul 29 2022 10:15AM,243057,10,8338063,MedStone Pharma LLC,Released
7,Jul 29 2022 10:15AM,243057,10,8338064,MedStone Pharma LLC,Released
8,Jul 29 2022 10:15AM,243057,10,8338065,MedStone Pharma LLC,Released
9,Jul 29 2022 10:15AM,243057,10,8338066,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,243054,Released,18
25,243056,Executing,1
26,243057,Released,11
27,243058,Released,1
28,243060,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243054,NaN,18.0
243056,1.0,NaN
243057,NaN,11.0
243058,NaN,1.0
243060,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242976,0.0,1.0
243003,0.0,1.0
243005,0.0,1.0
243006,0.0,1.0
243008,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242976,0,1
243003,0,1
243005,0,1
243006,0,1
243008,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242976,0,1
1,243003,0,1
2,243005,0,1
3,243006,0,1
4,243008,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242976,,1
1,243003,,1
2,243005,,1
3,243006,,1
4,243008,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc."
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc."
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC
14,Jul 29 2022 9:58AM,243054,10,Bio-PRF
32,Jul 29 2022 9:57AM,243056,12,Hush Hush
33,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.
41,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC"
43,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC"
51,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc.",,1
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc.",,1
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc,,1
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC,,11
4,Jul 29 2022 9:58AM,243054,10,Bio-PRF,,18
5,Jul 29 2022 9:57AM,243056,12,Hush Hush,1,
6,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,,8
7,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",,2
8,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",,8
9,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc.",1,
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc.",1,
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc,1,
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC,11,
4,Jul 29 2022 9:58AM,243054,10,Bio-PRF,18,
5,Jul 29 2022 9:57AM,243056,12,Hush Hush,,1
6,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,8,
7,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",2,
8,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",8,
9,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc.",1,
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc.",1,
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc,1,
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC,11,
4,Jul 29 2022 9:58AM,243054,10,Bio-PRF,18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc.",1.0,NaN
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc.",1.0,NaN
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc,1.0,NaN
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC,11.0,NaN
4,Jul 29 2022 9:58AM,243054,10,Bio-PRF,18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 29 2022 10:53AM,243060,19,"GCH Granules USA, Inc.",1.0,0.0
1,Jul 29 2022 10:52AM,243020,20,"ACI Healthcare USA, Inc.",1.0,0.0
2,Jul 29 2022 10:19AM,243058,10,Beach Products Inc,1.0,0.0
3,Jul 29 2022 10:15AM,243057,10,MedStone Pharma LLC,11.0,0.0
4,Jul 29 2022 9:58AM,243054,10,Bio-PRF,18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1944388,113.0,0.0
102,243045,1.0,0.0
12,486080,0.0,2.0
15,729144,19.0,0.0
16,243003,1.0,0.0
19,243060,1.0,0.0
20,972108,19.0,7.0
21,729019,3.0,0.0
22,972046,3.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1944388,113.0,0.0
1,102,243045,1.0,0.0
2,12,486080,0.0,2.0
3,15,729144,19.0,0.0
4,16,243003,1.0,0.0
5,19,243060,1.0,0.0
6,20,972108,19.0,7.0
7,21,729019,3.0,0.0
8,22,972046,3.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,113.0,0.0
1,102,1.0,0.0
2,12,0.0,2.0
3,15,19.0,0.0
4,16,1.0,0.0
5,19,1.0,0.0
6,20,19.0,7.0
7,21,3.0,0.0
8,22,3.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,113.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,19.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21,22
Status,,,,,,,,,
Executing,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,1.0
Released,113.0,1.0,0.0,19.0,1.0,1.0,19.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21,22
0,Executing,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,1.0
1,Released,113.0,1.0,0.0,19.0,1.0,1.0,19.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21,22
0,Executing,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,1.0
1,Released,113.0,1.0,0.0,19.0,1.0,1.0,19.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()